# Logistic Regression Training
Binary classification algorithm.
Typically used when the target variable has only two possible outcomes.

However, logistic regression can be extended to handle multi-class classification problems through techniques such as *one-vs-all* or *multinomial* logistic regression.

## One-vs-all or One-vs-Rest (OVR)
Train separate binary logistic regression model for each class.
Each model is trained to distinguish that class from all other classes.
During prediction, we run each observation through all models, and the class with the highest probability is assigned as the predicted class.

Our dataset has a discrete number of possible outcomes: `[Ravenclaw, Slytherin, Gryffindor, Hufflepuff]`.

This method allows breaking down by splitting up into multiple binary class models.

We will be using `k=4` *binary classifiers*.

## Features Selection
Based on data visualization:
- `Arithmancy` and `Care of magical Creatures` cannot classify well.
- `Defense Against the Dark Arts` and `Astronomy` are anti-correlated; we can drop one.

All other numerical features will be used for training.

## Data preparation
- Only meaningful features will be used
- Remove rows containing `NaN`
- *Standardize* data

## TODO
- use numpy to load and write to file
- Data preparation
    - Drop `Arithmancy`, `Care of Magical Creatures` and `Defense Against the Dark Arts`
    - Drop rows containing `NaN`
    - Standardize